In [1]:
import os
os.environ['PROJ_LIB'] = '/home/jlee/.conda/envs/mmc_sgp/share/proj'

import xarray as xr
import wrf
from netCDF4 import Dataset

file_dir = '/home/jlee/mmcpnnl/lasso_bundles/raw_wrfstat/'
file = file_dir+'wrfstat3_d01_2018-09-01_12:00:00.nc'

out_dir = file_dir+'trim/'
wip_file = out_dir+'test.nc'
out_file = out_dir+'out.nc'

# drop data below x meters
height_thres = 500

# wrf_single = Dataset(file, 'r')

In [2]:
csp_head = 'CSP_'
csp_var = ['Z', 'U', 'V', 'P', 'THV', 'TH', 'QV', 'QC', 'U2', 'V2', 'UW', 'VW', 'THV2', 'QV2', 
           'THDT_LS', 'QVDT_LS', 'UDT_LS', 'VDT_LS', 'W', 'W2']
csp_list = [csp_head+i for i in csp_var]

cst_head = 'CST_'
cst_var = ['UST', 'FSNS', 'FLNS', 'SH', 'LH', 'PS', 'PRECT', 'CLDTOT', 'LWP']
cst_list = [cst_head+i for i in cst_var]

# all_var_list = csp_list.extend(['XLAT', 'XLONG', 'XTIME', 'Times'])
all_var_list = csp_list+cst_list+['XLAT', 'XLONG', 'XTIME', 'Times']

In [3]:
ds = xr.open_dataset(file)

# create dummy XLAT XLONG
ds = ds.assign(XLAT=lambda ds: ds.CSS_LWP)
ds = ds.assign(XLONG=lambda ds: ds.CSS_LWP)

In [4]:
if os.path.isfile(wip_file):
    os.remove(wip_file)

ds[all_var_list].to_netcdf(wip_file)

ds_wip = ds[all_var_list]

In [5]:
wrft = Dataset(wip_file, 'r')

csp_w = wrf.getvar(wrft, 'CSP_W', timeidx=wrf.ALL_TIMES)
csp_w2 = wrf.getvar(wrft, 'CSP_W2', timeidx=wrf.ALL_TIMES)

csp_w_arr = wrf.destagger(csp_w, 1)
csp_w2_arr = wrf.destagger(csp_w2, 1)

ds_wip['CSP_W'] = (('Time', 'bottom_top'), csp_w_arr)
ds_wip['CSP_W2'] = (('Time', 'bottom_top'), csp_w2_arr)

ds_out = ds_wip.sel(bottom_top=slice(0, ds_wip['CSP_Z'].where(ds_wip['CSP_Z'][1, :] < 500, drop=True).shape[1]))

In [6]:
if os.path.isfile(out_file):
    os.remove(out_file)

ds_out.to_netcdf(out_file)

In [7]:
ds_wip['CSP_Z'].shape

(91, 226)

In [12]:
for var in csp_list[:3]: 
    print(var)
    print(ds_wip[var].shape)
    ds_wip[var] = ds_wip[var].where(ds_wip['CSP_Z'][1, :] < 500, drop=True)
    print(ds_wip[var].shape)

CSP_Z
(91, 226)


ValueError: arguments without labels along dimension 'bottom_top' cannot be aligned because they have different dimension sizes: {16, 226}

In [13]:
var

'CSP_Z'

In [28]:
ds_wip['CSP_Z'].where(ds_wip['CSP_Z'][1, :] < 500, drop=True).shape[1]

16

In [29]:
ds_wip.sel(bottom_top=slice(0, ds_wip['CSP_Z'].where(ds_wip['CSP_Z'][1, :] < 500, drop=True).shape[1]))

<xarray.Dataset>
Dimensions:      (Time: 91, bottom_top: 16, south_north: 250, west_east: 250)
Dimensions without coordinates: Time, bottom_top, south_north, west_east
Data variables:
    CSP_Z        (Time, bottom_top) float32 0.0 0.0 0.0 ... 418.5 449.1 479.7
    CSP_U        (Time, bottom_top) float32 0.0 0.0 0.0 ... 4.695 4.697 4.698
    CSP_V        (Time, bottom_top) float32 0.0 0.0 0.0 ... 5.261 5.269 5.277
    CSP_P        (Time, bottom_top) float32 0.0 0.0 0.0 ... 9.278e+04 9.246e+04
    CSP_THV      (Time, bottom_top) float32 0.0 0.0 0.0 ... 309.6 309.6 309.6
    CSP_TH       (Time, bottom_top) float32 0.0 0.0 0.0 ... 307.3 307.3 307.3
    CSP_QV       (Time, bottom_top) float32 0.0 0.0 0.0 ... 0.01242 0.01242
    CSP_QC       (Time, bottom_top) float32 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    CSP_U2       (Time, bottom_top) float32 0.0 0.0 0.0 ... 0.2233 0.2193 0.216
    CSP_V2       (Time, bottom_top) float32 0.0 0.0 0.0 ... 0.2335 0.2293 0.2253
    CSP_UW       (Time, bottom_top) float32 0.0 0.0 0.0 ... -0.006543 -0.006026
    CSP_VW       (Time, bottom_top) float32 0.0 0.0 0.0 ... 0.001424 0.002096
    CSP_THV2     (Time, bottom_top) float32 0.0 0.0 0.0 ... 0.0003734 0.0003628
    CSP_QV2      (Time, bottom_top) float32 0.0 0.0 0.0 ... 1.121e-09 1.069e-09
    CSP_THDT_LS  (Time, bottom_top) float32 0.0 0.0 0.0 ... -8.109e-05 -8.7e-05
    CSP_QVDT_LS  (Time, bottom_top) float32 0.0 0.0 ... -3.655e-08 -3.672e-08
    CSP_UDT_LS   (Time, bottom_top) float32 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    CSP_VDT_LS   (Time, bottom_top) float32 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    CSP_W        (Time, bottom_top) float32 0.0 0.0 ... -0.0002419 -0.0002559
    CSP_W2       (Time, bottom_top) float32 0.0 0.0 0.0 ... 0.2395 0.254 0.2667
    CST_UST      (Time) float32 0.0 0.1841 0.1811 ... 0.09161 0.08412 0.07691
    CST_FSNS     (Time) float32 0.0 -0.08619 -7.721 -25.38 ... 0.0 0.0 0.0 0.0
    CST_FLNS     (Time) float32 0.0 140.1 55.95 57.23 ... 73.63 73.73 73.81
    CST_SH       (Time) float32 0.0 -3.565 2.243 8.051 ... -36.57 -36.38 -36.19
    CST_LH       (Time) float32 0.0 19.63 25.58 31.52 ... 10.3 10.11 9.925 9.74
    CST_PS       (Time) float32 0.0 9.745e+04 9.744e+04 ... 9.741e+04 9.741e+04
    CST_PRECT    (Time) float32 0.0 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    CST_CLDTOT   (Time) float32 0.0 0.0 0.0 0.0 ... 0.233 0.3305 0.3482 0.2887
    CST_LWP      (Time) float32 0.0 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    XLAT         (Time, south_north, west_east) float32 ...
    XLONG        (Time, south_north, west_east) float32 ...
    XTIME        (Time) datetime64[ns] 2018-09-01T12:00:00 ... 2018-09-02T03:...
    Times        (Time) |S19 b'2018-09-01_12:00:00' ... b'2018-09-02_03:00:00'
Attributes:
    TITLE:                                  OUTPUT FROM WRF V3.8.1 MODEL
    START_DATE:                            2018-09-01_12:00:00
    WEST-EAST_GRID_DIMENSION:              251
    SOUTH-NORTH_GRID_DIMENSION:            251
    BOTTOM-TOP_GRID_DIMENSION:             227
    DX:                                    100.0
    DY:                                    100.0
    GRIDTYPE:                              C
    DIFF_OPT:                              2
    KM_OPT:                                2
    DAMP_OPT:                              3
    DAMPCOEF:                              0.2
    KHDIF:                                 1.0
    KVDIF:                                 1.0
    MP_PHYSICS:                            8
    RA_LW_PHYSICS:                         4
    RA_SW_PHYSICS:                         4
    SF_SFCLAY_PHYSICS:                     1
    SF_SURFACE_PHYSICS:                    1
    BL_PBL_PHYSICS:                        0
    CU_PHYSICS:                            0
    SF_LAKE_PHYSICS:                       0
    SURFACE_INPUT_SOURCE:                  3
    SST_UPDATE:                            0
    GRID_FDDA:                             0
    GFDDA_INTERVAL_M:                   

In [1]:
ds_wip['CSP_V']

NameError: name 'ds_wip' is not defined

In [13]:
ds_wip['CSP_P']

<xarray.DataArray 'CSP_P' (Time: 91, bottom_top: 226)>
array([[    0.    ,     0.    ,     0.    , ...,     0.    ,     0.    ,
            0.    ],
       [97445.01  , 97104.3   , 96765.45  , ..., 14101.315 , 13414.076 ,
        12957.203 ],
       [97443.21  , 97102.484 , 96763.64  , ..., 14101.341 , 13414.111 ,
        12957.208 ],
       ...,
       [97415.51  , 97075.26  , 96737.02  , ..., 14101.32  , 13414.104 ,
        12957.205 ],
       [97414.89  , 97074.64  , 96736.41  , ..., 14101.322 , 13414.104 ,
        12957.208 ],
       [97414.39  , 97074.11  , 96735.89  , ..., 14101.325 , 13414.1045,
        12957.208 ]], dtype=float32)
Dimensions without coordinates: Time, bottom_top
Attributes:
    FieldType:    104
    MemoryOrder:  Z  
    description:  Pressure
    units:        Pa
    stagger:

In [11]:
ds_wip['CSP_W']

<xarray.DataArray 'CSP_W' (Time: 91, bottom_top: 226)>
array([[ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [ 3.3150056e-05,  7.1569841e-05,  7.6167315e-05, ...,
        -1.4037765e-03, -1.7575035e-03, -1.9029260e-03],
       [ 7.6395954e-06,  2.1886242e-05,  2.2258017e-05, ...,
        -1.6000520e-03, -1.3242794e-03, -1.1569856e-03],
       ...,
       [-4.1262992e-05, -1.1035438e-04, -1.5463933e-04, ...,
        -2.1878958e-03, -2.1478541e-03, -2.0998716e-03],
       [-3.9495528e-05, -1.0474368e-04, -1.4755872e-04, ...,
        -2.1439139e-03, -2.0794384e-03, -2.0112395e-03],
       [-3.8029444e-05, -1.0081426e-04, -1.4269078e-04, ...,
        -2.1004858e-03, -2.0061582e-03, -1.9175124e-03]], dtype=float32)
Dimensions without coordinates: Time, bottom_top

In [96]:
ds['CSP_Z'].where(ds['CSP_Z'][1, :] < 500, drop=True)

<xarray.DataArray 'CSP_Z' (Time: 91, bottom_top: 16)>
array([[  0.       ,   0.       ,   0.       , ...,   0.       ,
          0.       ,   0.       ],
       [ 15.368521 ,  46.045296 ,  76.627594 , ..., 412.8449   ,
        443.38837  , 473.87982  ],
       [ 15.385088 ,  46.07692  ,  76.65514  , ..., 412.84723  ,
        443.3873   , 473.87555  ],
       ...,
       [ 15.6456785,  47.036476 ,  78.497025 , ..., 422.04727  ,
        452.96802  , 483.83994  ],
       [ 15.623034 ,  46.98251  ,  78.43192  , ..., 421.95352  ,
        452.87222  , 483.74225  ],
       [ 15.601221 ,  46.93007  ,  78.368004 , ..., 421.8604   ,
        452.777    , 483.64502  ]], dtype=float32)
Dimensions without coordinates: Time, bottom_top
Attributes:
    FieldType:    104
    MemoryOrder:  Z  
    description:  Half level height
    units:        m
    stagger:

In [114]:
ds

<xarray.Dataset>
Dimensions:         (Time: 91, bottom_top: 226, bottom_top_stag: 227, south_north: 250, south_north_stag: 251, west_east: 250, west_east_stag: 251)
Dimensions without coordinates: Time, bottom_top, bottom_top_stag, south_north, south_north_stag, west_east, west_east_stag
Data variables:
    Times           (Time) |S19 b'2018-08-05_12:00:00' ... b'2018-08-06_03:00...
    XTIME           (Time) datetime64[ns] 2018-08-05T12:00:00 ... 2018-08-06T...
    CST_CLDLOW      (Time) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    CST_CLDTOT      (Time) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    CST_LWP         (Time) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    CST_IWP         (Time) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    CST_PRECW       (Time) float32 0.0 32.78 32.78 32.79 ... 36.26 36.27 36.27
    CST_TKE         (Time) float32 0.0 1.891 4.374 15.51 ... 777.8 767.9 756.5
    CST_TSAIR       (Time) float32 0.0 296.6 296.8 297.1 ... 302.0 301.5 301.1
    CST_PS          (Time) float32 0.0 9.775e+04 ... 9.779e+04 9.779e+04
    CST_PRECT       (Time) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    CST_SH          (Time) float32 0.0 8.23 15.32 22.41 ... -37.75 -37.75 -37.76
    CST_LH          (Time) float32 0.0 28.5 34.58 40.66 ... 16.79 16.09 15.38
    CST_FSNTC       (Time) float32 0.0 -62.34 -96.42 -131.5 ... 0.0 0.0 0.0 0.0
    CST_FSNT        (Time) float32 0.0 -62.34 -96.42 -131.5 ... 0.0 0.0 0.0 0.0
    CST_FLNTC       (Time) float32 0.0 303.5 288.8 289.1 ... 294.9 294.6 294.2
    CST_FLNT        (Time) float32 0.0 303.5 288.8 289.1 ... 294.9 294.6 294.2
    CST_FSNSC       (Time) float32 0.0 -29.45 -50.87 -74.45 ... 0.0 0.0 0.0 0.0
    CST_FSNS        (Time) float32 0.0 -29.45 -50.87 -74.45 ... 0.0 0.0 0.0 0.0
    CST_FLNSC       (Time) float32 0.0 143.2 58.08 58.79 ... 69.69 69.56 69.36
    CST_FLNS        (Time) float32 0.0 143.2 58.08 58.79 ... 69.69 69.56 69.36
    CST_SWINC       (Time) float32 0.0 85.67 130.0 174.6 ... 0.0 0.0 0.0 0.0
    CST_TSK         (Time) float32 0.0 295.1 295.4 295.7 ... 302.0 301.7 301.5
    CST_UST         (Time) float32 0.0 0.1944 0.1623 ... 0.05645 0.054 0.05145
    CSP_Z           (Time, bottom_top) float32 0.0 0.0 ... 1.444e+04 1.472e+04
    CSP_Z8W         (Time, bottom_top_stag) float32 0.0 0.0 ... 1.485e+04
    CSP_DZ8W        (Time, bottom_top) float32 0.0 0.0 0.0 ... 299.3 299.8 255.4
    CSP_U           (Time, bottom_top) float32 0.0 0.0 0.0 ... 4.835 5.853 4.544
    CSP_V           (Time, bottom_top) float32 0.0 0.0 0.0 ... 0.605 2.727
    CSP_W           (Time, bottom_top_stag) float32 0.0 0.0 ... -0.001218
    CSP_P           (Time, bottom_top) float32 0.0 0.0 ... 1.347e+04 1.302e+04
    CSP_TH          (Time, bottom_top) float32 0.0 0.0 0.0 ... 368.0 369.9 372.1
    CSP_THV         (Time, bottom_top) float32 0.0 0.0 0.0 ... 368.0 369.9 372.1
    CSP_THL         (Time, bottom_top) float32 0.0 0.0 0.0 ... 368.0 369.9 372.1
    CSP_QV          (Time, bottom_top) float32 0.0 0.0 0.0 ... 2.999e-06 3e-06
    CSP_QC          (Time, bottom_top) float32 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    CSP_QI          (Time, bottom_top) float32 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    CSP_QL          (Time, bottom_top) float32 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    CSP_QF          (Time, bottom_top) float32 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    CSP_QT          (Time, bottom_top) float32 0.0 0.0 0.0 ... 2.999e-06 3e-06
    CSP_LWC         (Time, bottom_top) float32 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    CSP_IWC         (Time, bottom_top) float32 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    CSP_SPEQV       (Time, bottom_top) float32 0.0 0.0 ... 6.749e-07 6.547e-07
    CSP_A_CL        (Time, bottom_top) float32 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    CSP_RHO         (Time, bottom_top) float32 0.0 0.0 0.0 ... 0.225 0.2182
    CSP_U2          (Time, bottom_top) float32 0.0 0.0 ... 0.0005748 0.001047
    CSP_V2          (Time, bottom_top) float32 0.0 0.0 0.0 ... 0.002

In [41]:
wrft

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    TITLE:  OUTPUT FROM WRF V3.8.1 MODEL
    START_DATE: 2018-08-05_12:00:00
    WEST-EAST_GRID_DIMENSION: 251
    SOUTH-NORTH_GRID_DIMENSION: 251
    BOTTOM-TOP_GRID_DIMENSION: 227
    DX: 100.0
    DY: 100.0
    GRIDTYPE: C
    DIFF_OPT: 2
    KM_OPT: 2
    DAMP_OPT: 3
    DAMPCOEF: 0.2
    KHDIF: 1.0
    KVDIF: 1.0
    MP_PHYSICS: 8
    RA_LW_PHYSICS: 4
    RA_SW_PHYSICS: 4
    SF_SFCLAY_PHYSICS: 1
    SF_SURFACE_PHYSICS: 1
    BL_PBL_PHYSICS: 0
    CU_PHYSICS: 0
    SF_LAKE_PHYSICS: 0
    SURFACE_INPUT_SOURCE: 3
    SST_UPDATE: 0
    GRID_FDDA: 0
    GFDDA_INTERVAL_M: 0
    GFDDA_END_H: 0
    GRID_SFDDA: 0
    SGFDDA_INTERVAL_M: 0
    SGFDDA_END_H: 0
    HYPSOMETRIC_OPT: 1
    USE_THETA_M: 1
    WEST-EAST_PATCH_START_UNSTAG: 1
    WEST-EAST_PATCH_END_UNSTAG: 250
    WEST-EAST_PATCH_START_STAG: 1
    WEST-EAST_PATCH_END_STAG: 251
    SOUTH-NORTH_PATCH_START_UNSTAG: 1
    SOUTH-NORTH_PATCH_END_UNSTA

In [27]:
wrf.getvar(ds, 'XTIME', timeidx=wrf.ALL_TIMES)

TypeError: iteration over a 0-d array

In [23]:
wrf_single.variables

{'Times': <class 'netCDF4._netCDF4.Variable'>
 |S1 Times(Time, DateStrLen)
 unlimited dimensions: Time
 current shape = (91, 19)
 filling on, default _FillValue of   used,
 'XTIME': <class 'netCDF4._netCDF4.Variable'>
 float32 XTIME(Time)
     FieldType: 104
     MemoryOrder: 0  
     description: minutes since 2018-08-05 12:00:00
     units: minutes since 2018-08-05 12:00:00
     stagger: 
 unlimited dimensions: Time
 current shape = (91,)
 filling on, default _FillValue of 9.969209968386869e+36 used,
 'CST_CLDLOW': <class 'netCDF4._netCDF4.Variable'>
 float32 CST_CLDLOW(Time)
     FieldType: 104
     MemoryOrder: 0  
     description: Fractional low-cloud cover (<5 km)
     units: (0-1)
     stagger: 
 unlimited dimensions: Time
 current shape = (91,)
 filling on, default _FillValue of 9.969209968386869e+36 used,
 'CST_CLDTOT': <class 'netCDF4._netCDF4.Variable'>
 float32 CST_CLDTOT(Time)
     FieldType: 104
     MemoryOrder: 0  
     description: Fractional cloud cover
     units: (

In [3]:
wrf_single

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    TITLE:  OUTPUT FROM WRF V3.8.1 MODEL
    START_DATE: 2018-08-05_12:00:00
    WEST-EAST_GRID_DIMENSION: 251
    SOUTH-NORTH_GRID_DIMENSION: 251
    BOTTOM-TOP_GRID_DIMENSION: 227
    DX: 100.0
    DY: 100.0
    GRIDTYPE: C
    DIFF_OPT: 2
    KM_OPT: 2
    DAMP_OPT: 3
    DAMPCOEF: 0.2
    KHDIF: 1.0
    KVDIF: 1.0
    MP_PHYSICS: 8
    RA_LW_PHYSICS: 4
    RA_SW_PHYSICS: 4
    SF_SFCLAY_PHYSICS: 1
    SF_SURFACE_PHYSICS: 1
    BL_PBL_PHYSICS: 0
    CU_PHYSICS: 0
    SF_LAKE_PHYSICS: 0
    SURFACE_INPUT_SOURCE: 3
    SST_UPDATE: 0
    GRID_FDDA: 0
    GFDDA_INTERVAL_M: 0
    GFDDA_END_H: 0
    GRID_SFDDA: 0
    SGFDDA_INTERVAL_M: 0
    SGFDDA_END_H: 0
    HYPSOMETRIC_OPT: 1
    USE_THETA_M: 1
    WEST-EAST_PATCH_START_UNSTAG: 1
    WEST-EAST_PATCH_END_UNSTAG: 250
    WEST-EAST_PATCH_START_STAG: 1
    WEST-EAST_PATCH_END_STAG: 251
    SOUTH-NORTH_PATCH_START_UNSTAG: 1
    SOUTH-NORTH_PATCH_END_UNSTA

In [22]:
wrf_single.createVariable("lat","f4",("west_east","south_north"))

RuntimeError: NetCDF: Write to read only

In [15]:
wrf_single['CSP_UW']

<class 'netCDF4._netCDF4.Variable'>
float32 CSP_UW(Time, bottom_top)
    FieldType: 104
    MemoryOrder: Z  
    description: x-momentum flux uw (rs+sgs)
    units: m^2/s^2
    stagger: 
unlimited dimensions: Time
current shape = (91, 226)
filling on, default _FillValue of 9.969209968386869e+36 used

In [10]:
wrf_single['XTIME']

<class 'netCDF4._netCDF4.Variable'>
float32 XTIME(Time)
    FieldType: 104
    MemoryOrder: 0  
    description: minutes since 2018-08-05 12:00:00
    units: minutes since 2018-08-05 12:00:00
    stagger: 
unlimited dimensions: Time
current shape = (91,)
filling on, default _FillValue of 9.969209968386869e+36 used

In [14]:
wrf.getvar(wrf_single, 'XTIME', timeidx=wrf.ALL_TIMES)

<xarray.DataArray 'XTIME' (Time: 91)>
array([  0.,  10.,  20.,  30.,  40.,  50.,  60.,  70.,  80.,  90., 100.,
       110., 120., 130., 140., 150., 160., 170., 180., 190., 200., 210.,
       220., 230., 240., 250., 260., 270., 280., 290., 300., 310., 320.,
       330., 340., 350., 360., 370., 380., 390., 400., 410., 420., 430.,
       440., 450., 460., 470., 480., 490., 500., 510., 520., 530., 540.,
       550., 560., 570., 580., 590., 600., 610., 620., 630., 640., 650.,
       660., 670., 680., 690., 700., 710., 720., 730., 740., 750., 760.,
       770., 780., 790., 800., 810., 820., 830., 840., 850., 860., 870.,
       880., 890., 900.])
Dimensions without coordinates: Time
Attributes:
    FieldType:    104
    MemoryOrder:  0  
    description:  minutes since 2018-08-05 12:00:00
    units:        minutes since 2018-08-05 12:00:00
    stagger:

In [19]:
wrf_single['XLAT'] = 10

TypeError: 'netCDF4._netCDF4.Dataset' object does not support item assignment

In [24]:
wrf.getvar(wrf_single, 'CSP_UW', timeidx=wrf.ALL_TIMES)

ValueError: ('XLAT', 'XLAT_M') are not valid variable names